In [1]:
from keras.models import load_model
import cv2
import numpy as np
import pyttsx3
import matplotlib.pyplot as plt
import tkinter as tk
from PIL import Image, ImageTk

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

font_scale = 1  # Modified the font scale to be smaller
font = cv2.FONT_HERSHEY_PLAIN

# Initialize text-to-speech engine
engine = pyttsx3.init()

def start_detection():
    # Initialize the camera
    camera = cv2.VideoCapture(0)
    if not camera.isOpened():
        print("Unable to open camera")
        return

    # Create a new window for the camera display
    camera_window = tk.Toplevel()
    camera_window.title("Camera")
    camera_window.configure(bg="#E1E1E1")

    # Create a frame for the image
    image_frame = tk.Frame(camera_window, bg="white", width=800, height=600)
    image_frame.pack(pady=20)

    # Create a label for displaying the image
    image_label = tk.Label(image_frame)
    image_label.pack()

    def detect_objects():
        nonlocal camera
        # Grab the webcam image
        ret, image = camera.read()

        # Resize the raw image to (224-height, 224-width) pixels
        image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

        # Make the image a numpy array and reshape it to the model's input shape
        input_image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

        # Normalize the image array
        input_image = (input_image / 127.5) - 1

        # Predict the model
        prediction = model.predict(input_image)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        # Print prediction and confidence score
        print("Class:", class_name[2:], end="")
        print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")

        # Frame the detected object and show the label
        startX, startY, endX, endY = 112 - 50, 112 - 50, 112 + 50, 112 + 50  # A 100x100 rectangle in the center of the image
        cv2.rectangle(image, (startX, startY), (endX, endY), (255, 0, 0), 2)
        cv2.putText(image, f"{class_name[2:]}: {confidence_score * 100:.2f}%", (5, startY - 10),
                    font, fontScale=font_scale, color=(0, 255, 0), thickness=2)

        # Convert the OpenCV image to PIL format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_pil = Image.fromarray(image)

        # Create a PhotoImage object from the PIL image
        image_tk = ImageTk.PhotoImage(image_pil)

        # Update the image label with the new image
        image_label.configure(image=image_tk)
        image_label.image = image_tk

        # Activate voice output to announce the object and its meaning
        engine.say(f"I detected {class_name[2:]} with a confidence score of {confidence_score * 100:.2f} percent.")
        engine.runAndWait()

        # Call detect_objects again after a delay (10 milliseconds)
        image_label.after(10, detect_objects)

    def stop_detection():
        nonlocal camera
        # Release the camera resources
        camera.release()
        # Destroy the camera window
        camera_window.destroy()

    # Create the stop button
    stop_button = tk.Button(camera_window, text="Stop Detection", command=stop_detection, font=("Arial", 14), padx=10, pady=5, bg="#F44336", fg="white")
    stop_button.pack(pady=10)

    # Apply styling options
    camera_window.configure(bg="#E1E1E1")
    image_frame.configure(bd=2, relief=tk.SOLID)

    # Start object detection
    detect_objects()

# Create the main window
window = tk.Tk()
window.title("Object Detection")
window.geometry("900x700")
window.configure(bg="#E1E1E1")

# Create a frame for the buttons
button_frame = tk.Frame(window, bg="#E1E1E1")
button_frame.pack(pady=20)

# Create the start button
start_button = tk.Button(button_frame, text="Start Detection", command=start_detection, font=("Arial", 14), padx=10, pady=5, bg="#4CAF50", fg="white")
start_button.pack(side="left", padx=10)

# Apply styling options
window.configure(bg="#E1E1E1")
button_frame.configure(bd=2, relief=tk.SOLID)

# Start the Tkinter event loop
window.mainloop()

1/1 [==============================] - 2s 2s/step
Class:  tree
Confidence Score: 37 %
1/1 [==============================] - 0s 73ms/step
Class: bag
Confidence Score: 63 %
1/1 [==============================] - 0s 78ms/step
Class: bag
Confidence Score: 70 %
1/1 [==============================] - 0s 63ms/step
Class: bag
Confidence Score: 71 %
1/1 [==============================] - 0s 70ms/step
Class:  cat
Confidence Score: 67 %
1/1 [==============================] - 0s 76ms/step
Class: person
Confidence Score: 43 %
1/1 [==============================] - 0s 61ms/step
Class:  cat
Confidence Score: 56 %
1/1 [==============================] - 0s 62ms/step
Class:  cat
Confidence Score: 43 %
1/1 [==============================] - 0s 76ms/step
Class:  cat
Confidence Score: 86 %
